In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect = True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
latest_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
latest_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
end_date = dt.date(2017, 8, 23)

# Calculate the date one year from the last date in data set.
start_date = end_date - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
query = session.query(measurement.date, measurement.prcp)\
    .filter(func.strftime("%Y-%m-%d", measurement.date) >= start_date)\
    .all()\

# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_df = pd.DataFrame(query)
prcp_df["date"] = pd.to_datetime(prcp_df['date'])
prcp_df.set_index("date", inplace=True)

# Sort the dataframe by date
prcp_df.sort_index(inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data
prcp_df.plot(figsize=(12,8))
plt.ylabel("Inches", labelpad=30)
plt.xlabel("Date", labelpad=30)
plt.xticks(rotation="vertical", ha="center")
plt.legend(["Precipitation"])
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(station).count()
print(f"Total Stations: {station_count}")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(measurement.station, func.count(measurement.station))\
    .group_by(measurement.station)\
    .order_by(func.count(measurement.station).desc())\
    .all()
active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
tobs=[
    measurement.station,
    func.min(measurement.tobs),
    func.max(measurement.tobs),
    func.avg(measurement.tobs)
    ]

top_active_station = session.query(*tobs).filter(measurement.station == "USC00519281").all()

print(f"Most active station: {top_active_station[0][0]}")
print(f"Lowest Temperature: {top_active_station[0][1]}")
print(f"Highest temperature: {top_active_station[0][2]}")
print(f"Average Temperature: {top_active_station[0][3]}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
query = session.query(measurement.station, measurement.date, measurement.tobs)\
    .filter(func.strftime("%Y-%m-%d", measurement.date) > start_date)\
    .filter(measurement.station == "USC00519281")\
    .all()\

active_station_df = pd.DataFrame(query, columns=["Station", "Date", "Temperature"])
active_station_df.head()

In [ ]:
active_station_df.plot(kind="hist", bins=12, figsize=(16,9))
plt.xlabel("Temperature (°F)")
plt.ylabel("Frequency")
plt.show()

# Close session

In [ ]:
# Close Session
session.close()